In [ ]:
# core stuff
import pandas as pd

# Ensure that changes in imported module (gravann most importantly) are autoreloaded
%load_ext autoreload
%autoreload 2

In [ ]:
# Constants for conversion to metric
L = {
    "Bennu.pk": (352.1486930549145**2),
    "Churyumov-Gerasimenko.pk": 3126.6064453124995**2, 
    "Eros.pk" : 20413.864850997925**2,
    "Itokawa.pk": 350.438691675663**2,
    "Torus.pk": 3126.6064453124995**2,
    "Hollow.pk": 3126.6064453124995**2
}
conversion_unit = {
    "Bennu.pk": (7.329e10   * 6.67430e-11  / 352.1486930549145**2),
    "Churyumov-Gerasimenko.pk": 9.982e12 * 6.67430e-11 / 3126.6064453124995**2, 
    "Eros.pk" : 6.687e15 * 6.67430e-11 / 20413.864850997925**2,
    "Itokawa.pk": 3.51e10 * 6.67430e-11 / 350.438691675663**2,
    "Torus.pk": 9.982e12 * 6.67430e-11 / 3126.6064453124995**2,
    "Hollow.pk": 9.982e12 * 6.67430e-11 / 3126.6064453124995**2
}

In [ ]:
conversion_unit

To compute validation results, use the `run_validation.py` script. This notebook is used for postprocessing results from that.

### Load the file and define conversion constants for conversion to metric

In [ ]:
file = "results/VALIDATION.csv"
results = pd.read_csv(file)
results

### Extract all desired values

In [ ]:
cleaned_df = results.drop(columns=["Unnamed: 0","Loss",
                                   "RMSE@Altitude_0","relRMSE@Altitude_0",
                                   "RMSE@Altitude_1","relRMSE@Altitude_1",
                                   "RMSE@Altitude_2","relRMSE@Altitude_2",
                                   "Normalized L1 Loss@High Altitude","Normalized Relative Component Loss@High Altitude",
                                   "RMSE@High Altitude","relRMSE@High Altitude",
                                   "Normalized L1 Loss@Low Altitude","Normalized Relative Component Loss@Low Altitude",
                                   "RMSE@Low Altitude","relRMSE@Low Altitude"])
cleaned_df

In [ ]:
import numpy as np
for idx in range(cleaned_df.shape[0]):
    alt_str = cleaned_df.iloc[idx]["Altitudes"].replace("[","").replace("]","")
    alts =  np.fromstring(alt_str, dtype=float, sep=" ")
    cleaned_df["Altitudes"].loc[idx] = alts * L[cleaned_df.iloc[idx]["Sample"]]
    
    for col in cleaned_df.columns:
        if col in ["Normalized L1 Loss@Altitude_0","Normalized L1 Loss@Altitude_1","Normalized L1 Loss@Altitude_2"]:
            cleaned_df[col].loc[idx] = cleaned_df[col].loc[idx] * conversion_unit[cleaned_df.iloc[idx]["Sample"]]
    
cleaned_df = cleaned_df.set_index("Sample")   
print(cleaned_df)
    

In [ ]:
cleaned_df = cleaned_df.reindex(["Bennu.pk","Churyumov-Gerasimenko.pk","Eros.pk","Itokawa.pk","Hollow.pk","Torus.pk"])
pd.set_option("precision", 4)
np.set_printoptions(precision=1)

In [ ]:
[print(_[0],"\t\t\t &","% 10.2E &"%_[1][0],"% 10.2E &"%_[1][1],"% 10.2E &"%_[1][2]) for _ in list(zip(cleaned_df.index,cleaned_df["Altitudes"].values))];

In [ ]:
print("Abs")
[print("% 10.2E"%_[1],"&") for _ in list(zip(cleaned_df.index,cleaned_df["Normalized L1 Loss@Altitude_0"].values))];
print()
[print("% 10.2E"%_[1],"&") for _ in list(zip(cleaned_df.index,cleaned_df["Normalized L1 Loss@Altitude_1"].values))];
print()
[print("% 10.2E"%_[1],"&") for _ in list(zip(cleaned_df.index,cleaned_df["Normalized L1 Loss@Altitude_2"].values))];
print()
print("Rel")
[print('{:.3%}'.format(_[1]),"&") for _ in list(zip(cleaned_df.index,cleaned_df["Normalized Relative Component Loss@Altitude_0"].values))];
print()
[print('{:.3%}'.format(_[1]),"&") for _ in list(zip(cleaned_df.index,cleaned_df["Normalized Relative Component Loss@Altitude_1"].values))];
print()
[print('{:.3%}'.format(_[1])) for _ in list(zip(cleaned_df.index,cleaned_df["Normalized Relative Component Loss@Altitude_2"].values))];